In [1]:
%pip install pandas
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Ejercicio 3: Mostrar el numcontactos minimo en Madrid

In [12]:
import pyspark
from pyspark.sql import SparkSession
from pymongo import MongoClient
import collections
import pandas as pd

# Crea una SparkSession
spark = SparkSession.builder \
    .appName("MongoSparkConnector2") \
    .getOrCreate()

# Conecta a MongoDB usando pymongo
client = MongoClient("mongodb://172.17.0.3:27017/")
db = client['docker']
collection = db['perfilesmongo']

# Extrae los datos desde MongoDB
mongo_data = list(collection.find())

#print(mongo_data)

# Convierte los datos a un DataFrame de pandas
pdf = pd.DataFrame(mongo_data)

# Elimina la columna '_id' si es necesario, ya que no es serializable por defecto en Spark
if '_id' in pdf.columns:
    pdf = pdf.drop(columns=['_id'])

# Convierte el DataFrame de pandas a un DataFrame de Spark
df = spark.createDataFrame(pdf)

# Convertir el DataFrame a un RDD
rdd = df.rdd
#df.show()

rdd = df.select("location", "numcontactos").rdd

rdd_filtrado = rdd.filter(lambda x: "Madrid" in x[0])

locationcont = rdd_filtrado.map(lambda x: (x[0],x[1]))

minNumcontactos = locationcont.reduceByKey(lambda x, y: max(x , y))

results = minNumcontactos.collect()

for result in results:

    print(result[0] +" , " + str(result[1]))



Community of Madrid, Spain , 913
Greater Madrid Metropolitan Area , 999
Madrid , 997
